###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c)2014 L.A. Barba, C.D. Cooper, G.F. Forsyth.

###### With modifications by Andrea Viceré, Urbino University

## Sod's test problems

Sod's test problems are standard benchmarks used to assess the accuracy of numerical solvers. The tests use a classic example of one-dimensional compressible flow: the shock-tube problem. Sod (1978) chose initial conditions and numerical discretization parameters for the shock-tube problem and used these to test several schemes, including Lax-Wendroff and MacCormack's. Since then, many others have followed Sod's example and used the same tests on new numerical methods.

The shock-tube problem is so useful for testing numerical methods because it is one of the few problems that allows an exact solution of the Euler equations for compressible flow.

This notebook complements the previous lessons, particularly 03.05 and 05.01 with Sod's test problems as an independent coding exercise. We'll lay out the problem for you, but leave important bits of code for you to write on your own. Good luck!

### What's a shock tube?

A shock tube is an idealized device that generates a one-dimensional shock wave in a **compressible** gas. The setting allows an analytical solution of the Euler equations, which is very useful for comparing with the numerical results to assess their accuracy. 

Picture a tube with two regions containing gas at different pressures, separated by an infinitely-thin, rigid diaphragm. The gas is initially at rest, and the left region is at a higher pressure than the region to the right of the diaphragm. At time $t = 0.0 s$, the diaphragm is ruptured instantaneously.  

What happens?  

You get a shock wave.  The gas at high pressure, no longer constrained by the diaphragm, rushes into the lower-pressure area and a one-dimensional unsteady flow is established, consisting of:

* a shock wave traveling to the right
* an expansion wave traveling to the left
* a moving contact discontinuity

The shock-tube problem is an example of a *Riemann problem* and it has an analytical solution, as we said. The situation is illustrated in Figure 1.

![shocktube](shocktube.png)
#### Figure 1. The shock-tube problem.

### The Euler equations

The Euler equations govern the motion of an inviscid fluid (no viscosity). They consist of the conservation laws of mass and momentum, and often we also need to work with the energy equation. 

Let's consider a 1D flow with velocity $u$ in the $x$-direction. The Euler equations for a fluid with density $\rho$ and pressure $p$ are:

\begin{align}
\frac{\partial \rho}{\partial t} + \frac{\partial}{\partial x}(\rho u) &= 0\qquad\mathrm{Mass\ conservation} \\
\frac{\partial}{\partial t}(\rho u) + \frac{\partial}{\partial x} (\rho u^2 + p)&=0\qquad\mathrm{Momentum\  conservation}
\end{align}

plus the Bernoulli equation, which we can write in this form:

\begin{equation}
\frac{\partial}{\partial t}(\rho E_T) + \frac{\partial}{\partial x} (\rho u E_T +p u)=0\qquad\mathrm{Energy\ conservation}
\end{equation}
where $E_T=E+u^2/2$ is the total energy per unit mass, equal to the internal energy plus the kinetic energy (per unit mass).

Written in vector form, you can see that the Euler equations bear a formal resemblance to the traffic-density equation that has been the focus of previous modules. Here is the vector representation of the Euler equation:

\begin{equation}
\frac{\partial }{\partial t} \underline{\mathbf{u}} + \frac{\partial }{\partial x} \underline{\mathbf{f}} = 0
\end{equation}

The big difference with our previous work is that the variables $\underline{\mathbf{u}}$ and $\underline{\mathbf{f}}$ are *vectors*.  If you review the lesson 02.03 about the Phugoid full model, you will recall that we can solve for several values at once using the vector form of an equation. In the Phugoid Module, it was an ODE—now we apply the same procedure to a PDE.  

Let's take a look at what $\underline{\mathbf{u}}$ and $\underline{\mathbf{f}}$ consist of.  

### The conservative form

Many works in the early days of computational fluid dynamics in the 1960s showed that using the conservation form of the Euler equations is more accurate for situations with shock waves.  And as you already saw, the shock-tube solutions do contain shocks.

The conserved variables $\underline{\mathbf{u}}$ for Euler's equations are

\begin{equation}\underline{\mathbf{u}} = \left[ \begin{array}{c}
\rho \\
\rho u \\
\rho e_T \\ 
\end{array} \right]\end{equation}

where $\rho$ is the density of the fluid, $u$ is the velocity of the fluid and $e_T = e + \frac{u^2}{2}$ is the specific total energy; $\underline{\mathbf{f}}$ is the flux vector:

\begin{equation}\underline{\mathbf{f}} = \left[ \begin{array}{c}
\rho u \\
\rho u^2 + p \\
(\rho e_T + p) u \\ \end{array} \right]
\end{equation}

where $p$ is the pressure of the fluid.

If we put together the conserved variables and the flux vector into our PDE, we get the following set of equations:

\begin{equation}
\frac{\partial}{\partial t}
\left[ \begin{array}{c}
\rho \\
\rho u \\
\rho e_T \\ 
\end{array} \right]
+ \frac{\partial}{\partial x}
\left[ \begin{array}{c}
\rho u \\
\rho u^2 + p \\
(\rho e_T + p) u\\ \end{array}
\right]
=0
\end{equation}

There's one major problem there.  We have 3 equations and 4 unknowns: $\rho, u, e, p$.  But there is a solution!  We can use an equation of state to calculate the pressure—in this case, we'll use the ideal gas law.

### Calculating the pressure

For an ideal gas, the equation of state is

$$e = e(\rho, p) = \frac{p}{(\gamma -1) \rho},$$

where $\gamma = \frac{C_p}{C_V} = 1.4$ is a reasonable value to model air

$$p = (\gamma -1)\rho e. $$ 

Recall from above that

$$e_T = e+\frac{1}{2} u^2$$

hence

$$e = e_T - \frac{1}{2}u^2.$$

Putting it all together, we arrive at an equation for the pressure

$$p = (\gamma -1)\left(\rho e_T - \frac{\rho u^2}{2}\right).$$

### Flux in terms of $\underline{\mathbf{u}}$

With the traffic model, the flux was a function of traffic density.  For the Euler equations, the three equations we have are coupled and the flux *vector* is a function of $\underline{\mathbf{u}}$, the vector of conserved variables:

$$\underline{\mathbf{f}} = f(\underline{\mathbf{u}})$$

In order to get everything squared away, we need to represent $\underline{\mathbf{f}}$ in terms of $\underline{\mathbf{u}}$.
We can introduce a little shorthand for the $\underline{\mathbf{u}}$ and $\underline{\mathbf{f}}$ vectors and define:


$$\underline{\mathbf{u}} = 
\left[ \begin{array}{c}
u_1 \\
u_2 \\
u_3 \\ 
\end{array} \right] =
\left[ \begin{array}{c}
\rho \\
\rho u \\
\rho e_T \\ 
\end{array} \right]$$

$$\underline{\mathbf{f}} = 
\left[ \begin{array}{c}
f_1 \\
f_2 \\
f_3 \\ \end{array} \right] =
\left[ \begin{array}{c}
\rho u \\
\rho u^2 + p \\
(\rho e_T + p) u\\ \end{array}
\right]
$$  


With a little algebraic trickery, we can represent the pressure vector using quantities from the $\underline{\mathbf{u}}$ vector.

$$p = (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1} \right)$$

Now that pressure can be represented in terms of $\underline{\mathbf{u}}$, the rest of $\underline{\mathbf{f}}$ isn't too difficult to resolve:

$$\underline{\mathbf{f}} = \left[ \begin{array}{c}
f_1 \\
f_2 \\
f_3 \\ \end{array} \right] =
\left[ \begin{array}{c}
u_2\\
\frac{u^2_2}{u_1} + (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1} \right) \\
\left(u_3 + (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1}\right) \right) \frac{u_2}{u_1}\\ \end{array}
\right]$$

## Test conditions

The first test proposed by Sod in his 1978 paper is as follows.  

In a tube spanning from $x = -10 \text{m}$ to $x = 10 \text{m}$ with the rigid membrane at $x = 0 \text{m}$, we have the following initial gas states:

$$\underline{IC}_L = \left[ \begin{array}{c}
\rho_L \\ u_L \\ p_L \\ \end{array}\right] = 
\left[ \begin{array}{c}
1\ kg/m^3 \\ 0\ m/s \\ 100\ kN/m^2 \\ \end{array}\right]$$

$$\underline{IC}_R = \left[ \begin{array}{c}
\rho_R \\ u_R \\ p_R \\ \end{array}\right] = 
\left[ \begin{array}{c}
0.125\ kg/m^3 \\ 0\ m/s \\ 10\ kN/m^2 \\ \end{array}\right]$$

where $\underline{IC}_L$ are the initial density, velocity and pressure on the left side of the tube membrane and $\underline{IC}_R$ are the initial density, velocity and pressure on the right side of the tube membrane.  

The analytical solution to this test for the velocity, pressure and density, looks like the plots in Figure 2.

![shock_analytic](shock_tube_.01.png)

#### Figure 2. Analytical solution for Sod's first test.

### The Richtmyer method

For this exercise, you will be using a new scheme called the Richtmyer method.  Like the MacCormack *predictor-corrector* method that we learned in lesson 03.06, Richtmyer is a *two-step method*, given by:

\begin{align}
\underline{\mathbf{u}}^{n+\frac{1}{2}}_{i+\frac{1}{2}} &= \frac{1}{2} \left( \underline{\mathbf{u}}^n_{i+1} + \underline{\mathbf{u}}^n_i \right) - 
\frac{\Delta t}{2 \Delta x} \left( \underline{\mathbf{f}}^n_{i+1} - \underline{\mathbf{f}}^n_i\right) \\
\underline{\mathbf{u}}^{n+1}_i &= \underline{\mathbf{u}}^n_i - \frac{\Delta t}{\Delta x} \left(\underline{\mathbf{f}}^{n+\frac{1}{2}}_{i+\frac{1}{2}} - \underline{\mathbf{f}}^{n+\frac{1}{2}}_{i-\frac{1}{2}} \right)
\end{align}


The flux vectors used in the second step are obtained by evaluating the flux functions on the output of the first step:

$$\underline{\mathbf{f}}^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \underline{\mathbf{f}}\left(\underline{\mathbf{u}}^{n+\frac{1}{2}}_{i+\frac{1}{2}}\right).$$

The first step is like a *predictor* of the solution: if you look closely, you'll see that we are applying a Lax-Friedrichs scheme here. The second step is a *corrector* that applies a leapfrog update. Figure 3 gives a sketch of the stencil for Richtmyer method, where the "intermediate time" $n+1/2$ will require a temporary variable in your code, just like we had in the MacCormack scheme.


![richtmyer](richtmyer.png)


#### Figure 3. Stencil of Richtmyer scheme.

## Assignment

Your mission is to calculate the pressure, density and velocity across the shock tube at time $t = 0.01 s$.

Use first the Richtmayer method, which is a **Finite Difference Method**. Check that the results *looks like* the analytical solution we've shown.

If you really want a `30 e Lode` use also the MUSCL scheme, which is the **Finite Volume Method** that we have learned in lesson 05.01; in that case compare the results.

Good luck!

## Reference

* Sod, Gary A. (1978), "A survey of several finite difference methods for systems of nonlinear hyperbolic conservation laws," *J. Comput. Phys.*, Vol. 27, pp. 1–31 DOI: [10.1016/0021-9991(78)90023-2](http://dx.doi.org/10.1016%2F0021-9991%2878%2990023-2) // [PDF from Uni-Tuebingen](http://www.tat.physik.uni-tuebingen.de/~kley/lehre/bhydro/sod-paper.pdf), checked Nov. 3 , 2016.

**Prima pagina della pubblicazione scientifica**:
![paper](paper.jpg)

**Apparato di test degli shock tube reali e particolare del diaframma impiegato all'Università di Ottawa, Canada:**
<table cellspacing="2" cellpadding="2" width="600" border="0">
<div align='center'>
<tbody>
<tr>
<td valign="top" width="300"><img src="shock_tube_ottawa.jpg" /> </td>
<td valign="top" width="300"><img src="diaframma.jpg" /> </td>
</tr>
</tbody>
</div>
</table>

In [ ]:
from IPython.display import Video
Video("Shock_tube_in_azione.mp4", width = 500, height = 300)

**National Institute of Standards and Technology**: agenzia del governo degli Stati Uniti d'America che si occupa della gestione delle tecnologie. Tra le varie mansioni, la promozione dell'economia americana attraverso la collaborazione con l'industria al fine di sviluppare standard, tecnologie e metodologie che favoriscano la produzione e il commercio, la diffusione delle previsioni meteo in tutto il nord America e la principale fornitura di materiali standardizzati.  
*Fonte*: https://www.nist.gov/video/dynamic-pressure-sensing-shock-tube 

In [ ]:
# Importazione delle librerie
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Impostazione delle caratteristiche del font per il plot
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.weight'] = 'bold'
rcParams['font.size'] = 13

In [ ]:
# Parametri iniziali
# Costanti
CFL = 0.5                        # condizione di stabilità CFL 
v = 500                          # velocità massima (m/s)
gamma = 1.4                      # coefficiente di dilatazione adiabatica (aria)

# Valori per la discretizzazione
nx = 100                         # numero di punti nella griglia spaziale
nt = 30                          # numero di punti nella griglia temporale
dx = 20. / nx                    # dimensione del grid step, rapporto tra metri (20) e numero di punti 
dt = CFL * (dx / v)              # dimensione del time step, regolato dalla condizione CFL

# Rappresentazione del tubo
U = np.ones((3, nx))             # array che raccoglie i parametri (velocità, pressione, densità) del tubo
x = np.linspace(-10, 10, nx)     # rappresentazione del tubo, che si estende da -10m e 10m, per il plot

In [ ]:
# Funzione wrapper per il calcolo delle condizioni iniziali del tubo
def InitialConditions(gamma, nx):
    """
       Funzione che racchiude il calcolo delle condizioni iniziali
       
       Parametri:
       ----------
       gamma: coefficiente di dilatazione adiabatica  (float)
       
       Risultati:
       ----------
       U: parametri del tubo a t = 0                  (array) 
    """    
    # tuple che rappresentano i valori dei parametri presi in considerazione a destra e sinistra del diaframma, t = 0
    # l'aria all'interno del tubo è in quiete (0 m/s) e la sezione sinistra ha una pressione più alta rispetto alla destra
    # IC: densità vel pressione                
    c_s = (1.,    0, 100000.)                            # condizioni iniziali nel lato sinistro (IC_L)
    c_d = (0.125, 0,  10000.)                            # condizioni iniziali nel lato destro   (IC_R)
    
    d = int(nx / 2)                                      # calcolo della posizione del diaframma, a metà del tubo
 
    U[:,:d] = calcolaU(c_s[0], c_s[1], c_s[2], gamma)    # modellazione del tubo, lato sinistro (p1)
    U[:,d:] = calcolaU(c_d[0], c_d[1], c_d[2], gamma)    # modellazione del tubo, lato destro   (p5)
    
    return U

$$\underline{IC}_L = \left[ \begin{array}{c}
\rho_L \\ u_L \\ p_L \\ \end{array}\right] = 
\left[ \begin{array}{c}
1\ kg/m^3 \\ 0\ m/s \\ 100\ kN/m^2 \\ \end{array}\right]$$

$$\underline{IC}_R = \left[ \begin{array}{c}
\rho_R \\ u_R \\ p_R \\ \end{array}\right] = 
\left[ \begin{array}{c}
0.125\ kg/m^3 \\ 0\ m/s \\ 10\ kN/m^2 \\ \end{array}\right]$$

In [ ]:
# Funzione che calcola i parametri iniziali di una zona del tubo
def calcolaU(rho, u, p, gamma):
    """
       Calcola i parametri iniziali di una zona del tubo
       
       Parametri:
       ----------
       rho: densità                                   (int)   
       u: velocità                                    (int)
       p: pressione                                   (int)
       gamma: coefficiente di dilatazione adiabatica  (float) 
       
       Risultati:
       ----------
       U: parametri di una zona del tubo a t = 0     (array) 
    """
    e = p / ((gamma - 1) * rho)                     # energia interna
    eT = e + u**2 / 2                               # energia totale per unità di massa
    
    return np.array([[rho], [rho * u], [rho * eT]]) # calcolo dei parametri a t = 0

$$e = e(\rho, p) = \frac{p}{(\gamma -1) \rho},$$

$$e_T = e+\frac{1}{2} u^2$$

$$\underline{\mathbf{u}} = 
\left[ \begin{array}{c}
u_1 \\
u_2 \\
u_3 \\ 
\end{array} \right] =
\left[ \begin{array}{c}
\rho \\
\rho u \\
\rho e_T \\ 
\end{array} \right]$$

In [ ]:
# Funzione che calcola la soluzione con il metodo Richtmyer
# Il primo step è lo step detto predittore e segue il modello Lax-Friedrichs
# Il secondo step è lo step detto correttore e segue il modello Leapfrog
def Richtmyer(U, dx, dt, nx, nt, gamma, norm = 0.0):
    """
       Calcola la soluzione con il metodo Richtmyer
       
       Parametri:
       ----------
       U: parametri iniziali del tubo                 (array)
       dx: dimensione del grid step                   (int)
       dt: dimensione del time step                   (int)
       nx: numero di punti nel tubo                   (int)
       nt: numero di punti nel tempo                  (int)
       gamma: coefficiente di dilatazione adiabatica  (float)
       norm: fattore di smorzamento (opzionale)       (float, compreso tra 0.0 e 0.2)
       
       Risultati:
       ----------
       U: parametri del tubo dopo lo shock            (array)
    """
    # array provvisori per il calcolo della soluzione
    UP = np.ones((3, nx))        # array intermedio
    UP_lf = np.ones((3, nx))     # array che rappresenta il primo step, Lax-Friedrichs
    UP_leap = np.ones((3, nx))   # array che rappresenta il secondo step, Leapfrog
    
    for i in range(nt):

        # primo step, Lax-Friedrichs
        UP_lf[:,:-1] = 0.5 * (U[:,1:] + U[:,:-1]) - dt / (2 * dx) * (calcolaF(U[:,1:], gamma) - calcolaF(U[:,:-1], gamma))
        
        # secondo step, Leapfrog
        # il metodo Leapfrog non è self starting, il primo step deve essere fornito da un'altro metodo
        UP_leap[:,1:] = UP_lf[:,:-1] 
        UP[:,1:-1] = U[:,1:-1] - dt/dx * (calcolaF(UP_lf[:,1:-1], gamma) - calcolaF(UP_leap[:,1:-1], gamma)) +\
        norm * (U[:,2:] - 2 * U[:,1:-1] + U[:,:-2])  #normalizzazione 
        
        # memorizzazione dei risultati
        UP[:,0] = UP[:,1]
        UP[:,-1] = UP[:,-2]
        U[:,:] = UP.copy()
        
    return U

\begin{align}
\underline{\mathbf{u}}^{n+\frac{1}{2}}_{i+\frac{1}{2}} &= \frac{1}{2} \left( \underline{\mathbf{u}}^n_{i+1} + \underline{\mathbf{u}}^n_i \right) - 
\frac{\Delta t}{2 \Delta x} \left( \underline{\mathbf{f}}^n_{i+1} - \underline{\mathbf{f}}^n_i\right) \\
\underline{\mathbf{u}}^{n+1}_i &= \underline{\mathbf{u}}^n_i - \frac{\Delta t}{\Delta x} \left(\underline{\mathbf{f}}^{n+\frac{1}{2}}_{i+\frac{1}{2}} - \underline{\mathbf{f}}^{n+\frac{1}{2}}_{i-\frac{1}{2}} \right)
\end{align}

\begin{align}
\underline{\mathbf{f}}^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \underline{\mathbf{f}}\left(\underline{\mathbf{u}}^{n+\frac{1}
{2}}_{i+\frac{1}{2}}\right)
\end{align}

In [ ]:
# Funzione che calcola il flusso in funzione dei valori noti di U
def calcolaF(U, gamma):
    """
       Calcola il flusso in funzione di U
       
       Parametri:
       ----------
       U: parametri del tubo                          (array)
       gamma: coefficiente di dilatazione adiabatica  (float)
       
       Risultati:
       ----------
       F: flusso                                      (array) 
    """
    # variabili provvisorie per velocitò, pressione e densità
    u1, u2, u3 = U[0], U[1], U[2]
    
    # calcolo vettoriale del flusso
    return np.array([u2,
                     u2**2 / u1 + (gamma - 1) * (u3 - u2**2 / (2 * u1)),
                     u2 / u1 * (u3 + (gamma - 1) * (u3 - u2**2 / (2 * u1)))])

$$\underline{\mathbf{f}} = \left[ \begin{array}{c}
f_1 \\
f_2 \\
f_3 \\ \end{array} \right] =
\left[ \begin{array}{c}
u_2\\
\frac{u^2_2}{u_1} + (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1} \right) \\
\left(u_3 + (\gamma -1)\left(u_3 - \frac{1}{2} \frac{u^2_2}{u_1}\right) \right) \frac{u_2}{u_1}\\ \end{array}
\right]$$

In [ ]:
# Funzione che ricava velocità, pressione e densità dalla soluzione
def RicavaRisultati(U, gamma):
    """
       Ricava i valori di velocità, pressione, e densità dal tubo dopo lo shock
       
       Parametri:
       ----------
       U: parametri del tubo dopo lo shock            (array)
       gamma: coefficiente di dilatazione adiabatica  (float)
       
       Risultati:
       ----------
       vel: velocità                                  (int)
       pres: pressione                                (int)
       den: densità                                   (int)
    """
    
    # ricavo dei risultati
    vel = U[1,:] / U[0,:]
    pres = (gamma - 1) * (U[2,:] - 0.5 * U[1,:]**2 / U[0,:])
    den = U[0,:]
    
    return vel, pres, den

In [ ]:
# Funzione per il plot dei risultati, suddivisi in velocità, pressione e densità
def PlotShockTube(vel, pres, rho, x, colore, t):
    # dimensione del grafico
    plt.figure(figsize = (18, 12))
    
    # velocità
    plt.subplot(2, 3, 1)
    plt.plot(x, vel, color = colore, ls = '-', lw = 2)
    plt.title('Velocità a t = %.2f s' % t)
    plt.xlabel('Metri')
    plt.ylabel('$m/s$')
    plt.xticks(range(-10, 11, 5))
    plt.yticks(range(0, 401, 50), rotation = 45)
    plt.axhline(y = 0, color = '#000000', ls = ':')
    
    # pressione
    plt.subplot(2, 3, 2)
    plt.plot(x, pres, color = colore, ls = '-', lw = 2)
    plt.title('Pressione a t = %.2f s' % t)
    plt.xlabel('Metri')
    plt.ylabel('$N/m^2$')
    plt.xticks(range(-10, 11, 5))
    plt.yticks(range(0, 100001, 10000), rotation = 45)
    plt.axhline(y = 10000, color = '#000000', ls = ':')
    
    # densità
    plt.subplot(2, 3, 3)
    plt.plot(x, den, color = colore, ls = '-', lw = 2)
    plt.title('Densità a t = %.2f s' % t)
    plt.xlabel('Metri')
    plt.ylabel('$kg/m^3$')
    plt.xticks(range(-10, 11, 5))
    plt.yticks(np.arange(0.0, 1.1, 0.1), rotation = 45)
    plt.axhline(y = 0.12, color = '#000000', ls = ':')

In [ ]:
# Condizioni iniziali
# Il gas è in quiete (0 m/s), la parte sinistra del tubo ha pressione e densità maggiore
U = InitialConditions(gamma, nx)
vel, pres, den, = RicavaRisultati(U, gamma)
PlotShockTube(vel, pres, den, x, '#FFA500', 0)

In [ ]:
# Soluzione numerica standard
# Il diaframma viene aperto e il gas si muove, generando uno shock
U = InitialConditions(gamma, nx)
U = Richtmyer(U, dx, dt, nx, nt, gamma)
vel, pres, den, = RicavaRisultati(U, gamma)
PlotShockTube(vel, pres, den, x, '#FF0000', 0.01)

In [ ]:
# Soluzione normalizzata
U = InitialConditions(gamma, nx)
U = Richtmyer(U, dx, dt, nx, nt, gamma, norm = 0.1)
vel, pres, den, = RicavaRisultati(U, gamma)
PlotShockTube(vel, pres, den, x, '#0000FF', 0.01)

**Soluzione analitica**:
![shock_analytic](shock_tube_.01.png)

**Discussione dei risultati:** Possiamo notare che nel calcolo della soluzione con il ***metodo Richtmyer***, nei punti in cui la funzione non è regolare (**punti di discontinuità**) sono presenti delle oscillizazioni numeriche.  Normalizzando i risultati con una media mobile, le oscillazioni scompaiono ma le curve non sono così definite come nella soluzione standard. Nonostante ciò, il ***metodo Richtmyer*** rimane un valido modello poichè, come tutti i **modelli predittore-correttore**, è di secondo ordine (introduce un errore **O(Δt<sup>2</sup>)**), facile da implementare e che non richiede molti calcoli (come invece avviene per esempio nel caso del modello *Lax-Wendroff*, il quale richiede ad ogni time step il calcolo dei *Jacobiani*) Inoltre, poichè lo step predittore è direzionato verso la propagazione dello shock, il modello viene rappresentato in maniera più accurata rispetto al caso in cui i due step fossero invertiti.

In [ ]:
# Funzione che calcola la soluzione con il metodo Richtmyer con gli step invertiti
# Il primo step è lo step detto predittore e segue il modello Leapfrog
# Il secondo step è lo step detto correttore e segue il modello Lax-Friedrichs
def Richtmyer_inv(U, dx, dt, nx, nt, gamma):
    """
       Calcola la soluzione con il metodo Richtmyer con gli step invertiti
       
       Parametri:
       ----------
       U: parametri iniziali del tubo                 (array)
       dx: dimensione del grid step                   (int)
       dt: dimensione del time step                   (int)
       nx: numero di punti nel tubo                   (int)
       nt: numero di punti nel tempo                  (int)
       gamma: coefficiente di dilatazione adiabatica  (float)
       
       Risultati:
       ----------
       U: parametri del tubo dopo lo shock            (array)
    """
    # array provvisori per il calcolo della soluzione
    UP = np.ones((3, nx))        # array intermedio
    UP_leap = np.ones((3, nx))   # array che rappresenta il primo step, Leapfrog
    UP_lf = np.ones((3, nx))     # array che rappresenta il secondo step, Lax-Friedrichs
    
    for i in range(nt):
        
        # primo step, Leapfrog
        # il metodo Leapfrog non è self starting, il primo step deve essere fornito da un'altro metodo
        UP_leap[:,1:] = 0.5 * (U[:,1:] + U[:,:-1]) - dt / (2 * dx) * (calcolaF(U[:,1:], gamma) - calcolaF(U[:,:-1], gamma))
        UP[:,1:-1] = U[:,1:-1] - dt/dx * (calcolaF(UP_lf[:,1:-1], gamma) - calcolaF(UP_leap[:,1:-1], gamma))
        
        # secondo step, Lax-Friedrichs
        UP[:,:-1] = 0.5 * (UP_leap[:,1:] + UP_leap[:,:-1]) - dt / (2 * dx) * (calcolaF(UP_leap[:,1:], gamma) -\
        calcolaF(UP_leap[:,:-1], gamma))
        
        # memorizzazione dei risultati
        UP[:,0] = UP[:,1]
        UP[:,-1] = UP[:,-2]
        U[:,:] = UP.copy()
        
    return U

In [ ]:
# Soluzione con i due step dello schema predittore-correttore invertiti
U = InitialConditions(gamma, nx)
U = Richtmyer_inv(U, dx, dt, nx, nt, gamma)
vel, pres, den, = RicavaRisultati(U, gamma)
PlotShockTube(vel, pres, den, x, '#000000', 0.01)

Studente: Alessio Muzi  
Matricola n°: 299329  
Sessione Invernale, Simulazione Numerica  
Prof. Andrea Vicerè  